In [1]:
import pandas as pd 
import os 
from src.utils import sample_process
from src.utils.text_processing import normalizer, jaccard_similarity_2_shingle


In [2]:
def get_description(df, column: list[str] = ["Tên thiết bị", "Số quản lý thiết bị", "Ngày phát sinh", "Nắm bắt hiện tượng", 
                                             "Nguyên nhân 1", "Nguyên nhân 2", "Nguyên nhân gốc", "Xử lý", "Nội dung phòng chống tái phát"]):
    """ 
    return the description of the machine error
    """
    res = ""
    for idx, row in df[column].iterrows():
        res += f"Lỗi thứ {idx + 1}\n"
        for col in column:
            res += f"{col}: {row[col]}\n" if row[col] is not None else ""
        res += "\n"
    return res 

In [3]:
maintain_df = pd.read_csv("data\maintain.csv")
maintain_df.head()

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\acer\AppData\Local\Temp\ipykernel_14988\2724576126.py:1: SyntaxWarning: invalid escape sequence '\m'
  maintain_df = pd.read_csv("data\maintain.csv")


,Trạng thái,Số chỉ thị,Line,Tên thiết bị,Số quản lý thiết bị,Loại công trình,PP bảo dưỡng,Vùng thao tác,LK đồng bộ,LK không thể tháo rời,...,Nguyên nhân 2,Nguyên nhân gốc,Xử lý,Nội dung phòng chống tái phát,Ngày PCTP,Nội dung chỉ đạo,Model linh kiện,Tên linh kiện,Vị trí xuất kho,Lượng xuất kho
0,Đóng,H013173,B4,OP4,VMI 0082,BM,21.0,20.0,BM27,MZ15,...,nghi ngờ do mòn lacking trục vít Y ở vị trí đổ...,NaN,--- 1: bù điều chỉnh góc orien --- 2: Kiểm tr...,"nên kế hoạch thay thế trục vít Y1, Y2 , kiểm t...",05-01-2024,NaN,NaN,NaN,NaN,NaN
1,Đóng,H013215,B4,OP4,VMI 0082,BM,21.0,2.0,BM6,MD1,...,không dầu cấp vào cụm ATC.,lỗi cấp dầu.,--- 1: kiểm tra post giữ dao. --- 2: Kiểm tra...,"thay kiểm tra, thay thế linh kiện cụm ATC.",06-01-2024,NaN,NaN,NaN,NaN,NaN
2,Đóng,H013271,B4,OP4,VMI 0082,BM,21.0,20.0,BM6,MD1,...,"không có dầu bôi trơn , dầu bôi trơn bị lão hóa",mòn seal trục ATC,--- 1: thay tay ATC --- 2: cải tạo cụm gear ATC,"lên kế hoạch kiểm tra độ rơ cụm ATC , dầu bôi ...",07-01-2024,"- Kiểm tra lại toàn bộ tay ATC đã rebuild, khắ...",●P/#3610510(số bản vẽ: D/#0490033A),ATC ARM ASSY,NaN,1.0
3,Đóng,H013273,B2,OP6,VMI 0061,BM,21.0,5.0,BE3,EC2,...,I/O luôn cấp điện cho role đóng phanh,Tiếp điểm IO hỏng,--- 1: thay IO điều khiển đóng mở role phanh ...,nên kế hoạch kiểm tra,08-01-2024,NaN,P/#T700000 (SBD-P124-S01),BRAKE UNIT,NaN,1.0
4,Đóng,H013357,B6,OP2,VMI 0125,BM,21.0,20.0,BM14,MM1,...,NaN,NaN,--- 1: Kẹt cửa magazine --- 2: Cửa magazine t...,Tiến hành kiểm tra rà soát tất cả cửa magazine...,09-01-2024,NaN,NaN,NaN,NaN,NaN


In [4]:
# machine_id = "VMI 0082"
machine_id = None

machine_name = "OP4"
error_describe = "lỗi EX1000"

In [5]:

""" 
Question answerd: hãy cho tôi biết lỗi này lúc trc được giải quyết bằng cách nào 

Input: 
    machine_name : tên máy, 
    machine_id: mã máy
    error_describe: mô tả lõi ví dụ dao cắt bị mòn

Ouptut: thống kê các biện pháp đã được xử lý với máy

    Ví dụ: 
    
    Trong quá khứ, máy CNC mã máy: …. với lỗi đã được sửa chữa 5 lần bao gồm các biện pháp 
    
    - biện pháp 1
    - biện pháp 2
    - biện pháp 3

Idea: 
    - truy xuất thông tin lỗi trong dataframe
    - dùng 
    """
machine_data = maintain_df[maintain_df['Tên thiết bị'] == machine_name] if machine_name is not None else maintain_df

machine_data = maintain_df[maintain_df['Số quản lý thiết bị'] == machine_id ] if machine_id is not None else machine_data
machine_data

,Trạng thái,Số chỉ thị,Line,Tên thiết bị,Số quản lý thiết bị,Loại công trình,PP bảo dưỡng,Vùng thao tác,LK đồng bộ,LK không thể tháo rời,...,Nguyên nhân 2,Nguyên nhân gốc,Xử lý,Nội dung phòng chống tái phát,Ngày PCTP,Nội dung chỉ đạo,Model linh kiện,Tên linh kiện,Vị trí xuất kho,Lượng xuất kho
0,Đóng,H013173,B4,OP4,VMI 0082,BM,21.0,20.0,BM27,MZ15,...,nghi ngờ do mòn lacking trục vít Y ở vị trí đổ...,NaN,--- 1: bù điều chỉnh góc orien --- 2: Kiểm tr...,"nên kế hoạch thay thế trục vít Y1, Y2 , kiểm t...",05-01-2024,NaN,NaN,NaN,NaN,NaN
1,Đóng,H013215,B4,OP4,VMI 0082,BM,21.0,2.0,BM6,MD1,...,không dầu cấp vào cụm ATC.,lỗi cấp dầu.,--- 1: kiểm tra post giữ dao. --- 2: Kiểm tra...,"thay kiểm tra, thay thế linh kiện cụm ATC.",06-01-2024,NaN,NaN,NaN,NaN,NaN
2,Đóng,H013271,B4,OP4,VMI 0082,BM,21.0,20.0,BM6,MD1,...,"không có dầu bôi trơn , dầu bôi trơn bị lão hóa",mòn seal trục ATC,--- 1: thay tay ATC --- 2: cải tạo cụm gear ATC,"lên kế hoạch kiểm tra độ rơ cụm ATC , dầu bôi ...",07-01-2024,"- Kiểm tra lại toàn bộ tay ATC đã rebuild, khắ...",●P/#3610510(số bản vẽ: D/#0490033A),ATC ARM ASSY,NaN,1.0
10,Đóng,H014104,B5,OP4,VMI 0105,BM,21.0,3.0,BM6,MD1,...,nghi ngờ không có dầu bôi trơn dẫn đến mài mòn,lỗi thao tác kiểm tra cấp dầu,--- 1: kiểm tra độ rơ trục saft tay ATC --- 2...,"lên kế hoạch kiểm tra độ rơ trục saft , dầu ob...",20-01-2024,NaN,NaN,NaN,NaN,NaN
13,Đóng,H014637,B6,OP4,VMI 0127,BM,21.0,5.0,BE5,EF2,...,lão hóa tuổi thọ 5 năm,NaN,--- 1: thay AS xác nhận đóng cửa phải,thay AS mới,22-01-2024,NaN,D4C-1324,Limit Switch,NaN,1.0
16,Hoàn thành,H010263,B5,OP4,VMI 0105,BM,21.0,2.0,BM6,MD1,...,NaN,NaN,--- 1: dơi dao T4.,lên kế hoạch kiểm tra dầu cấp cụm ATC và thông...,24-01-2024,NaN,NaN,NaN,NaN,NaN
31,Đóng,H042740,B4,OP4,VMI 0082,BM,21.0,2.0,BA18,AZ15,...,"Do dầu đục, Đóng cặn dầu lâu ngày dẫn đến tắc ...",NaN,--- 1: Bình dầu AMO III DSL,"Vệ sinh Fillter, vệ sinh bình dầu, cấp dầu mới",22-04-2024,NaN,NaN,NaN,NaN,NaN
32,Đóng,H042885,A3,OP4,VMI 0091,BM,21.0,2.0,BA18,AZ15,...,NaN,NaN,--- 1: Rơ le phanh ATC,Thay thế các máy tương tự,26-04-2024,NaN,LY2N-D2 DC24V,POWER RELAY,NaN,1.0
33,Hoàn thành,H050334,B2,OP4,VMI 0059,BM,21.0,20.0,BA18,AZ15,...,NaN,NaN,"--- 1: thay telecop X,Y",NaN,NaN,"- Vệ sinh những vị trí tương tự, kéo dài tuổi ...",P/#3852726,XY Telescopic cover,NaN,1.0
45,Đóng,H060767,B4,OP4,VMI 0082,BM,21.0,0.0,BA18,AZ15,...,do mạt phanh mài mòn bám vào má phanh,NaN,--- 1: Phanh ATC --- 2: kiểm tra role phanh,Nghiên cứu vệ sinh mạt do mòn phanh,14-06-2024,NaN,NaN,NaN,NaN,NaN


In [6]:
if machine_data.empty:
    print( 'Không tìm thấy thông tin máy')



In [7]:
machine_data['sim_score'] = machine_data['Nắm bắt hiện tượng'].apply(lambda x: jaccard_similarity_2_shingle(x, error_describe))
machine_data

C:\Users\acer\AppData\Local\Temp\ipykernel_14988\992972007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine_data['sim_score'] = machine_data['Nắm bắt hiện tượng'].apply(lambda x: jaccard_similarity_2_shingle(x, error_describe))


,Trạng thái,Số chỉ thị,Line,Tên thiết bị,Số quản lý thiết bị,Loại công trình,PP bảo dưỡng,Vùng thao tác,LK đồng bộ,LK không thể tháo rời,...,Nguyên nhân gốc,Xử lý,Nội dung phòng chống tái phát,Ngày PCTP,Nội dung chỉ đạo,Model linh kiện,Tên linh kiện,Vị trí xuất kho,Lượng xuất kho,sim_score
0,Đóng,H013173,B4,OP4,VMI 0082,BM,21.0,20.0,BM27,MZ15,...,NaN,--- 1: bù điều chỉnh góc orien --- 2: Kiểm tr...,"nên kế hoạch thay thế trục vít Y1, Y2 , kiểm t...",05-01-2024,NaN,NaN,NaN,NaN,NaN,0.066667
1,Đóng,H013215,B4,OP4,VMI 0082,BM,21.0,2.0,BM6,MD1,...,lỗi cấp dầu.,--- 1: kiểm tra post giữ dao. --- 2: Kiểm tra...,"thay kiểm tra, thay thế linh kiện cụm ATC.",06-01-2024,NaN,NaN,NaN,NaN,NaN,0.058824
2,Đóng,H013271,B4,OP4,VMI 0082,BM,21.0,20.0,BM6,MD1,...,mòn seal trục ATC,--- 1: thay tay ATC --- 2: cải tạo cụm gear ATC,"lên kế hoạch kiểm tra độ rơ cụm ATC , dầu bôi ...",07-01-2024,"- Kiểm tra lại toàn bộ tay ATC đã rebuild, khắ...",●P/#3610510(số bản vẽ: D/#0490033A),ATC ARM ASSY,NaN,1.0,0.083333
10,Đóng,H014104,B5,OP4,VMI 0105,BM,21.0,3.0,BM6,MD1,...,lỗi thao tác kiểm tra cấp dầu,--- 1: kiểm tra độ rơ trục saft tay ATC --- 2...,"lên kế hoạch kiểm tra độ rơ trục saft , dầu ob...",20-01-2024,NaN,NaN,NaN,NaN,NaN,0.388889
13,Đóng,H014637,B6,OP4,VMI 0127,BM,21.0,5.0,BE5,EF2,...,NaN,--- 1: thay AS xác nhận đóng cửa phải,thay AS mới,22-01-2024,NaN,D4C-1324,Limit Switch,NaN,1.0,0.086957
16,Hoàn thành,H010263,B5,OP4,VMI 0105,BM,21.0,2.0,BM6,MD1,...,NaN,--- 1: dơi dao T4.,lên kế hoạch kiểm tra dầu cấp cụm ATC và thông...,24-01-2024,NaN,NaN,NaN,NaN,NaN,0.055556
31,Đóng,H042740,B4,OP4,VMI 0082,BM,21.0,2.0,BA18,AZ15,...,NaN,--- 1: Bình dầu AMO III DSL,"Vệ sinh Fillter, vệ sinh bình dầu, cấp dầu mới",22-04-2024,NaN,NaN,NaN,NaN,NaN,0.200000
32,Đóng,H042885,A3,OP4,VMI 0091,BM,21.0,2.0,BA18,AZ15,...,NaN,--- 1: Rơ le phanh ATC,Thay thế các máy tương tự,26-04-2024,NaN,LY2N-D2 DC24V,POWER RELAY,NaN,1.0,0.133333
33,Hoàn thành,H050334,B2,OP4,VMI 0059,BM,21.0,20.0,BA18,AZ15,...,NaN,"--- 1: thay telecop X,Y",NaN,NaN,"- Vệ sinh những vị trí tương tự, kéo dài tuổi ...",P/#3852726,XY Telescopic cover,NaN,1.0,0.035714
45,Đóng,H060767,B4,OP4,VMI 0082,BM,21.0,0.0,BA18,AZ15,...,NaN,--- 1: Phanh ATC --- 2: kiểm tra role phanh,Nghiên cứu vệ sinh mạt do mòn phanh,14-06-2024,NaN,NaN,NaN,NaN,NaN,0.583333


In [8]:
res  = machine_data.loc[machine_data['sim_score'] > 0.4]
res = res.sort_values(by='sim_score', ascending=False)



In [9]:
res

,Trạng thái,Số chỉ thị,Line,Tên thiết bị,Số quản lý thiết bị,Loại công trình,PP bảo dưỡng,Vùng thao tác,LK đồng bộ,LK không thể tháo rời,...,Nguyên nhân gốc,Xử lý,Nội dung phòng chống tái phát,Ngày PCTP,Nội dung chỉ đạo,Model linh kiện,Tên linh kiện,Vị trí xuất kho,Lượng xuất kho,sim_score
45,Đóng,H060767,B4,OP4,VMI 0082,BM,21.0,0.0,BA18,AZ15,...,NaN,--- 1: Phanh ATC --- 2: kiểm tra role phanh,Nghiên cứu vệ sinh mạt do mòn phanh,14-06-2024,NaN,NaN,NaN,NaN,NaN,0.583333


In [11]:
description = get_description(res)
print(description)

Lỗi thứ 46
Tên thiết bị: OP4
Số quản lý thiết bị: VMI 0082
Ngày phát sinh: 14-06-2024
Nắm bắt hiện tượng: #12771: Lỗi EX1004 ATC
Nguyên nhân 1: phanh ATC không đóng mở
Nguyên nhân 2: do mạt phanh mài mòn bám vào má phanh
Nguyên nhân gốc: nan
Xử lý:  --- 1: Phanh ATC --- 2: kiểm tra role phanh
Nội dung phòng chống tái phát: Nghiên cứu vệ sinh mạt do mòn phanh


